# Assignment 1: Processes and IPC

## Question 1: Effect of the location of os.wait()

### Code 1.1: Put os.wait() before the print command

In [ ]:
import os
from time import sleep

rc = os.fork()
if (rc > 0):
  os.wait()
  print('-> Print 1: I am the parent process (PID: %d) of \
  a child process (PID: %d)\n' %(os.getpid(), rc))
elif (rc == 0):
  sleep(5)
  print('-> Print 2:, I am a child process (PID: %d)\n' %os.getpid())
  os._exit(os.EX_OK)
else:
  print('Error!')


### Code 1.2: Put os.wait() after the print command

In [ ]:
import os
from time import sleep

rc = os.fork()
if (rc > 0):
  print('-> Print 1: I am the parent process (PID: %d) of \
  a child process (PID: %d)\n' %(os.getpid(), rc))
  os.wait()
elif (rc == 0):
  sleep(5)
  print('-> Print 2:, I am a child process (PID: %d)\n' %os.getpid())
  os._exit(os.EX_OK)
else:
  print('Error!')


## Question 2: How many processes will be created?

In [ ]:
import os

rc1 = os.fork()
if (rc1 > 0):
  print('-> PID: %d\n' %os.getpid())
  rc2 = os.fork()
  if (rc2 > 0):
    print('-> PID: %d\n' %os.getpid())
  elif (rc2 == 0):
    print('-> PID: %d\n' %os.getpid())
    os._exit(os.EX_OK)
elif (rc1 == 0):
  print('-> PID: %d\n' %os.getpid())
  rc3 = os.fork()
  if (rc3 > 0):
    print('-> PID: %d\n' %os.getpid())
    rc4 = os.fork()
    if (rc4 > 0):
      print('-> PID: %d\n' %os.getpid())
      os._exit(os.EX_OK)
    elif (rc4 == 0):
      print('-> PID: %d\n' %os.getpid())
      os._exit(os.EX_OK)
  elif (rc3 == 0):
    print('-> PID: %d\n' %os.getpid())
    os._exit(os.EX_OK)


## Question 3: Process creation using multiprocessing

In [ ]:
import os
from multiprocessing import Process

def PrintPID():
  print('Process PID = %d\n' %os.getpid())

def ChildTask1():
  PrintPID()
  Proc4 = Process(target=PrintPID)
  Proc5 = Process(target=PrintPID)
  Proc4.start()
  Proc5.start()
  Proc4.join()
  Proc5.join()

def ChildTask2():
  PrintPID()
  Proc6 = Process(target=PrintPID)
  Proc6.start()
  Proc6.join()

def ChildTask3():
  PrintPID()
  Proc7 = Process(target=PrintPID)
  Proc8 = Process(target=PrintPID)
  Proc9 = Process(target=PrintPID)
  Proc7.start()
  Proc8.start()
  Proc9.start()
  Proc7.join()
  Proc8.join()
  Proc9.join()

if __name__ == "__main__":
  Proc1 = Process(target=ChildTask1)
  Proc2 = Process(target=ChildTask2)
  Proc3 = Process(target=ChildTask3)
  Proc1.start()
  Proc2.start()
  Proc3.start()
  Proc1.join()
  Proc2.join()
  Proc3.join()
  PrintPID()


## Question 4: Speed up by using multiple processes and shared memory

### Code 4.1: A single process

In [ ]:
from time import time

if __name__ == "__main__":
  time0 = time()
  Num = 0
  for n in range(1, 60000001):
    Num = Num + n
  time1 = time()-time0
  print('Total sum = %d' %Num)
  print('Computation time = %f' %time1)


### Code 4.2: Multiple processes

In [ ]:
from multiprocessing import Process
from multiprocessing.shared_memory import SharedMemory
from time import time

def TaskAdd(a, b, MemName):
  Num = 0
  for n in range(a, b+1):
    Num = Num + n
  sm = SharedMemory(MemName)
  sm.buf[:11] = bytearray(Num.to_bytes(10, byteorder='big') )

if __name__ == "__main__":
  time0 = time()
  shared_mem1 = SharedMemory(name='MyMem1', create=True, size=10)
  shared_mem2 = SharedMemory(name='MyMem2', create=True, size=10)
  shared_mem3 = SharedMemory(name='MyMem3', create=True, size=10)
  Proc1 = Process(target=TaskAdd, args=(1, 20000000, 'MyMem1', ))
  Proc2 = Process(target=TaskAdd, args=(20000001, 40000000, 'MyMem2', ))
  Proc3 = Process(target=TaskAdd, args=(40000001, 60000000, 'MyMem3', ))
  Proc1.start()
  Proc2.start()
  Proc3.start()
  Proc1.join()
  Proc2.join()
  Proc3.join()
  data1 = shared_mem1.buf[:11]
  data2 = shared_mem2.buf[:11]
  data3 = shared_mem3.buf[:11]
  TotalSum = int.from_bytes(data1, "big") + int.from_bytes(data2, "big") + int.from_bytes(data3, "big")
  del data1
  del data2
  del data3
  shared_mem1.close()
  shared_mem2.close()
  shared_mem3.close()
  shared_mem1.unlink()
  shared_mem2.unlink()
  shared_mem3.unlink()
  time1 = time()-time0
  print('Total sum = %d' %TotalSum)
  print('Computation time = %f' %time1)


# End